# Анализ инвестиций в недвижимость России

## Краткое описание проекта

Данный проект анализирует возможности инвестирования в арендную недвижимость в 10 городах России:
- Сочи
- Краснодар
- Уфа
- Казань
- Екатеринбург
- Омск
- Тюмень
- Новосибирск
- Калининград
- Владивосток

### Основные результаты

- **Лучший ROI:** Омск (12%), Екатеринбург, Новосибирск (11%)
- **Быстрая окупаемость:** Инвестиции в Омске окупаются примерно за 9 лет
- **Анализ рисков:** Сочи имеет самые высокие цены на недвижимость, но самую низкую доходность

### Методология

В проекте реализован полный цикл анализа данных:
1. Сбор данных с платформы ЦИАН
2. Очистка и предобработка данных
3. Инженерия признаков
4. Статистический анализ
5. Визуализация результатов

## Импорт необходимых библиотек

Для анализа данных мы будем использовать следующие библиотеки:

In [1]:
# Основные библиотеки для анализа данных
import pandas as pd
import numpy as np

# Библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Библиотеки для веб-скрапинга
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup

# Дополнительные библиотеки
import requests
import re
import time
import random

# Импорт локального модуля
import cian_scraper as cs

## Сбор данных

Настроим параметры для сбора данных с сайта ЦИАН:

In [2]:
# Параметры поиска
cities = ['Сочи', 'Краснодар', 'Уфа', 'Казань', 'Екатеринбург', 'Омск', 'Тюмень', 'Новосибирск', 'Калининград', 'Владивосток']
cost_range = ['30000', '100000']  # Диапазон цен аренды
# cian url
url_cian = "https://www.cian.ru"
# Создаем DataFrame для хранения данных
columns = [
    'Объявление', 'Параметры', 'Стоимость', 'Условия аренды',
    'Адрес', 'Описание', 'Город', 'Ссылка'
]
all_data_df = pd.DataFrame(columns=columns)

### Процесс сбора данных

Запустим сбор данных по всем городам:

In [ ]:
# инициализация драйвера
driver = cs.initialize_browser()

In [3]:
try:
    for city in cities:
        # Реинициализируем DataFrame для каждого города
        df = pd.DataFrame(columns=['Объявление', 'Параметры', 'Стоимость', 'Условия аренды', 'Адрес', 'Описание', 'Ссылка'])
     
        s = cs.start_work(driver, city, url_cian, cost_range)
 
        # собираем данные с сайта – информация объявлений по 15 страницам.
        result_df = cs.parse_all_pages(driver, df, max_pages=15)
  
        # Добавляем информацию о городе
        result_df['Город'] = city
        
        # Добавляем в общий DataFrame
        all_data_df = pd.concat([all_data_df, result_df], ignore_index=True)

    # Сохраняем все результаты в один CSV
    all_data_df.to_csv('cian_listings_all_cities.csv', index=False, encoding='utf-8-sig')
    print(f"Данные успешно сохранены в файл 'cian_listings_all_cities.csv'. Всего объявлений: {len(all_data_df)}")

except Exception as e:
    print(f"Произошла ошибка при сборе данных: {str(e)}")

finally:
    # Закрываем браузер после обработки всех городов
    print("Закрываем браузер. Работа завершена.")
    driver.quit()


Город Сочи
Обработано страниц: 15
Всего собрано объявлений: 420

Город Краснодар
Обработано страниц: 15
Всего собрано объявлений: 420

Город Уфа
Обработано страниц: 15
Всего собрано объявлений: 417

Город Казань
Обработано страниц: 15
Всего собрано объявлений: 420

Город Екатеринбург
Обработано страниц: 15
Всего собрано объявлений: 420

Город Омск
Обработано страниц: 15
Всего собрано объявлений: 410

Город Тюмень
Обработано страниц: 15
Всего собрано объявлений: 420

Город Новосибирск
Обработано страниц: 15
Всего собрано объявлений: 420

Город Калининград
Обработано страниц: 15
Всего собрано объявлений: 395

Город Владивосток
Обработано страниц: 15
Всего собрано объявлений: 420
Данные успешно сохранены в файл 'cian_listings_all_cities.csv'. Всего объявлений: 4162

Закрываем браузер. Работа завершена.



## Загрузка и предобработка данных

Загрузим собранные данные из CSV файла и проведем их очистку:

In [4]:
# Загрузка данных
data = pd.read_csv('cian_listings_all_cities.csv')

# Базовая очистка данных
if __name__ == "__main__":
    # Очищаем адреса в датафрейме и создаём новые столбцы с микрорайоном и адресом
    data = cs.clean_addresses(data)
    # столбец стоимость обработка
    data['Стоимость'] = data['Стоимость'].apply(
        lambda x: int(x.split('₽/мес.')[0].replace(' ', '')) if '₽/мес.' in x else x
    )
    # переименование столбца
    data = data.rename(columns={'Стоимость': 'Стоимость. ₽/мес.'})
    # кодировка значений
    data = cs.process_real_data(data['Параметры'], data['Условия аренды'], data)
    # заполним пустое значение в столбце
    data['Микрорайон'] = data['Микрорайон'].replace('', pd.NA).fillna('Нет информации')
    # тип данных стоимости
    data['Стоимость 1 кв.м. (аренда)'] = data['Стоимость 1 кв.м. (аренда)'].astype(int)
    # очищаем данные
    data.drop_duplicates(keep='first', inplace=True)
    # убираем запятую в конце строки второго столбца
    data['Тип жилья'] = data['Тип жилья'].str.rstrip(',')

# удаляем дубликаты
data = data.drop(data.columns[[0, 1, 3]], axis=1)  # Удаляет первый и третий столбцы

In [5]:
data.head(3)

,Стоимость. ₽/мес.,Адрес,Описание,Город,Ссылка,Микрорайон,Тип жилья,Площадь,Этаж,Этажность дома,Срок аренды,Коммунальные включены,Счетчики включены,Комиссия,Залог,Стоимость 1 кв.м. (аренда)
0,30000,"улица Красный Путь, 143к5",АН Галеон представляет Вашему вниманию абсолют...,Омск,https://omsk.cian.ru/rent/flat/311363105/,Город Водников,1-комн. квартира,47.0,6.0,17.0,Долгосрочно,5000.0,False,70.0,35000.0,638
1,32000,"улица Перелета, 34",АН Галеон представляет Вашему вниманию прекрас...,Омск,https://omsk.cian.ru/rent/flat/306248532/,Прибрежный,2-комн. квартира,70.0,8.0,9.0,Краткосрочно,True,False,70.0,20000.0,457
2,35000,"набережная Иртышская, 33",Сдается двушка с шикарным видом на Иртыш.\nВ к...,Омск,https://omsk.cian.ru/rent/flat/317540446/,Нет информации,2-комн. квартира,41.0,5.0,9.0,Краткосрочно,True,False,0.0,35000.0,853


In [6]:
# корректируем столбец – тип жилья
replace_dict = {
    '1-комн. апартаменты': '1-комн. квартира',
    'Апартаменты-студия': 'Студия',
    '2-комн. апартаменты': '2-комн. квартира',
}
data['Тип жилья'] = data['Тип жилья'].replace(replace_dict)

In [7]:
# Порядок столбцов
cols_to_move = ['Город', 'Тип жилья', 'Площадь', 'Стоимость. ₽/мес.']
cols = [col for col in data.columns if col not in cols_to_move]
new_order = cols_to_move + cols
data = data[new_order]

## Собираем данные с интернет-ресурса о средней стоимости квадратного метра покупки жилья по городам (Россия)

In [9]:
if __name__ == "__main__":
    cost_per_meter_df = cs.get_cost_per_meter_data()
    if cost_per_meter_df is not None:
        print(f"Получено {len(cost_per_meter_df)} записей о стоимости покупки жилья.")
        cost_per_meter_df = cost_per_meter_df.iloc[:, :2]
        data = data.merge(cost_per_meter_df[['Город', cost_per_meter_df.columns[1]]], on='Город', how='left')
        print(f'Данные объединены (merge). Объем {data.shape[0]} строк, {data.shape[1]} столбцов')
# сохраняем для дальнейшего использования
cost_per_meter_df.to_csv('Стоимость_м2_Россия.csv')  

In [10]:
cost_per_meter_df = pd.read_csv('Стоимость_м2_Россия.csv', usecols=lambda column: column != 'Unnamed: 0')
data = data.merge(cost_per_meter_df[['Город', cost_per_meter_df.columns[1]]], on='Город', how='left')

In [11]:
# Преобразовываем числовые столбцы 

# 1. Округление столбцов до одного знака после запятой
columns_to_round = ['ROI', 'Срок окупаемости']
for col in columns_to_round:
    if col in data.columns:
        data[col] = data[col].round(1)  # 1 - количество знаков после запятой

# 2. Преобразование столбцов в целые числа (int)
columns_to_int = ['Площадь', 'Этаж', 'Этажность дома', 'Комиссия', 'Залог', 'Стоимость покупки', 'Стоимость 1 кв.м. (аренда)', 'Стоимость 1 кв.м. (покупка)']
for col in columns_to_int:
    if col in data.columns:
        # При преобразовании в int отсутствующие значения нужно обработать отдельно
        # Вариант 1: Заменить NaN на какое-то значение, например 0, и затем преобразовать
        data[col] = data[col].fillna(0).astype(int)

In [12]:
data.to_csv('data_f.csv')

In [13]:
display(data.head(2))

,Город,Тип жилья,Площадь,Стоимость. ₽/мес.,Адрес,Описание,Ссылка,Микрорайон,Этаж,Этажность дома,Срок аренды,Коммунальные включены,Счетчики включены,Комиссия,Залог,Стоимость 1 кв.м. (аренда),Стоимость 1 кв.м. (покупка)
0,Омск,1-комн. квартира,47,30000,"улица Красный Путь, 143к5",АН Галеон представляет Вашему вниманию абсолют...,https://omsk.cian.ru/rent/flat/311363105/,Город Водников,6,17,Долгосрочно,5000.0,False,70,35000,638,78515
1,Омск,2-комн. квартира,70,32000,"улица Перелета, 34",АН Галеон представляет Вашему вниманию прекрас...,https://omsk.cian.ru/rent/flat/306248532/,Прибрежный,8,9,Краткосрочно,True,False,70,20000,457,78515


In [14]:
# Получаем расчёты
result = cs.analyze_real_estate_investments(data)

# Получение каждого из значений
summary = result['summary']  # Сводная таблица
roi_by_type_city = result['roi_by_type_city']  # ROI по типам жилья и городам
top_roi = result['top_roi']  # Топ-5 предложений по ROI

In [15]:
summary_config = {
                'Средний ROI (%)': {'cmap': 'RdBu', 'vmin': 1.5, 'vmax': 13.5, 'format': '{:,.2f}'},#+
                'Средний срок окупаемости (лет)': {'cmap': 'OrRd', 'vmin': 6, 'vmax': 14, 'format': '{:,.2f}'}
}
# Применяем 
style_summary = cs.advanced_format_dataframe(summary, summary_config)

## Посмотрим на сводный отчёт 

In [16]:
display(style_summary)

,Город,Средняя стоимость (₽/мес.),Медиана стоимости (₽/мес.),Средняя площадь (м²),Средний ROI (%),Медиана ROI (%),Минимальный ROI (%),Максимальный ROI (%),Стандартное отклонение ROI,Средний срок окупаемости (лет),Медиана срок окупаемости (лет)
6,Омск,38921,35000,50,12,12,6,26,4,9.00,9
1,Екатеринбург,38946,35000,44,11,11,5,28,3,10.00,9
5,Новосибирск,37129,32000,41,11,11,5,30,4,10.00,9
3,Калининград,50205,45000,51,10,9,4,21,3,11.00,11
4,Краснодар,43329,35000,52,10,9,4,23,4,11.00,11
9,Уфа,39126,35000,49,10,10,5,32,4,10.00,10
8,Тюмень,38421,35000,51,9,9,4,28,3,12.00,11
0,Владивосток,43840,35000,44,8,6,4,28,5,15.00,16
2,Казань,42054,37000,51,8,7,3,21,3,14.00,14
7,Сочи,51614,45000,37,7,7,2,25,3,15.00,14


# Топ-5 предложений по ROI

In [17]:
columns_to_round_decimal = ['Площадь', 'ROI', 'Срок окупаемости']
top_roi[columns_to_round_decimal] = top_roi[columns_to_round_decimal].round(1)
top_roi[['Стоимость 1 кв.м. (покупка)']] = top_roi[['Стоимость 1 кв.м. (покупка)']].round(0).astype(int)

display(top_roi)

,Город,Тип жилья,Площадь,Стоимость. ₽/мес.,Стоимость 1 кв.м. (покупка),ROI,Срок окупаемости
1146,Уфа,Студия,25,65000,96529,32.3,3.1
2488,Новосибирск,2-комн. квартира,40,100000,100000,30.0,3.3
1033,Уфа,Студия,27,65000,96529,29.9,3.3
2431,Новосибирск,1-комн. квартира,31,76000,100000,29.4,3.4
2949,Владивосток,Студия,27,100000,157576,28.2,3.5


# ROI по типам жилья и городам

In [18]:
display(roi_by_type_city)

Тип жилья,1-комн. квартира,2-комн. квартира,Студия
Город,,,
Владивосток,8.53,6.54,17.54
Екатеринбург,11.90,9.69,15.69
Казань,8.02,7.19,11.17
Калининград,10.58,8.91,11.12
Краснодар,10.93,8.56,13.12
Новосибирск,11.57,9.68,14.07
Омск,14.42,10.56,16.15
Сочи,7.02,6.73,8.15
Тюмень,10.71,8.05,13.62


## Был сгенерирован отчёт PDF, был создан Dashboard

<div style="display: flex; align-items: center; flex-wrap: wrap; margin: 16px 0;">
 
  <a href="https://drive.google.com/file/d/1WqB5XILFmQJv-J_UNJSODtm5ht34r5Ec/view?usp=sharing" 
     target="_blank"
     style="display: inline-block; padding: 10px 16px; margin: 8px; font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Open Sans', 'Helvetica Neue', sans-serif; font-size: 14px; font-weight: 500; text-align: center; text-decoration: none; background-color: #ECF2F8; color: #2E4153; border: 2px solid #7C8A97; border-radius: 4px; cursor: pointer; transition: all 0.2s ease-in-out;"
     onmouseover="this.style.backgroundColor='#218838'; this.style.borderColor='#218838';"
     onmouseout="this.style.backgroundColor='#28a745'; this.style.borderColor='#28a745';">
    <span style="margin-right: 8px;">📉</span>Отчёт PDF
  </a>
  
  <a href="https://dash-app-1-8xh5.onrender.com/" 
     target="_blank"
     style="display: inline-block; padding: 10px 16px; margin: 8px; font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Open Sans', 'Helvetica Neue', sans-serif; font-size: 14px; font-weight: 500; text-align: center; text-decoration: none; background-color: #ECF2F8; color: #2E4153; border: 2px solid #7C8A97; border-radius: 4px; cursor: pointer; transition: all 0.2s ease-in-out;"
     onmouseover="this.style.backgroundColor='#218838'; this.style.borderColor='#218838';"
     onmouseout="this.style.backgroundColor='#28a745'; this.style.borderColor='#28a745';">
    <span style="margin-right: 8px;">📊</span>Панель анализа данных (dashboard)
  </a>
</div>

# Общая ситуация на рынке
На основе анализа рынка арендной недвижимости в 10 городах России можно сделать следующие выводы:

**Наиболее доходные города:** Омск, Екатеринбург и Новосибирск демонстрируют самую высокую среднюю доходность инвестиций (ROI) — 12%, 11% и 11% соответственно.

**Наименее доходные города:** Сочи (7%), Владивосток и Казань (по 8%) показывают наименьшую доходность.

**Средние сроки окупаемости:** В более доходных городах срок окупаемости составляет около 9-10 лет, в то время как в Сочи и Владивостоке он достигает 15 лет.

**Ценовая ситуация:** Несмотря на низкий ROI, Сочи и Владивосток имеют одни из самых высоких средних арендных ставок (51,614₽ и 43,840₽ соответственно).

## Наиболее доходные инвестиционные объекты
### Топ-5 объектов с максимальным ROI:

1. Студия в Уфе (25.8 м²) — ROI 31.4%, окупаемость 3.2 года
2. 2-комнатная квартира в Новосибирске (40 м²) — ROI 30%, окупаемость 3.3 года
3. Студия в Уфе (27.2 м²) — ROI 29.8%, окупаемость 3.4 года
4. 1-комнатная квартира в Новосибирске (31 м²) — ROI 29.4%, окупаемость 3.4 года
5. Студия во Владивостоке (27.1 м²) — ROI 28.1%, окупаемость 3.6 года

### Доходность по типам жилья

1. **Студии** показывают наибольшую доходность во всех городах (в среднем 13-17%)
2. **1-комнатные квартиры** занимают среднюю позицию по доходности (8-11%)
3. **2-комнатные квартиры** демонстрируют наименьший ROI (6-10%)

## Рекомендации для инвесторов

- **Наиболее перспективные объекты:** Студии в городах с более низкой стоимостью квадратного метра при покупке, но стабильным спросом на аренду (Уфа, Новосибирск)
- **Оптимальный баланс:** 1-комнатные квартиры обеспечивают разумный ROI при более низких рисках по сравнению со студиями
- **Географическая диверсификация:** Инвестирование в разные города может снизить региональные риски

### Оговорка о высокой изменчивости цен и ограничениях анализа
Важно учитывать следующие факторы:

>Данный анализ отражает текущий момент времени и не учитывает сезонных колебаний цен на аренду. 
Высокая волатильность рынка недвижимости может существенно изменить показатели ROI.
Не учтены дополнительные расходы (налоги, ремонт, управление недвижимостью).
Не рассматривались микролокации внутри городов, которые могут значительно влиять на привлекательность объектов.
Реальная доходность может отличаться из-за возможных простоев между арендаторами.
Исследование не учитывает динамику цен на недвижимость, которая также влияет на итоговый ROI инвестиций.